In [1]:
import numpy as np
from authentic_generator_np import DemandGenerator

/Users/konggullerod/Documents/Dokumenter/Skole - DTU/7. Semester/Bachelor projekt/Model/authentic_generator_np.py:38: SyntaxWarning: invalid escape sequence '\i'
  """


Parameters
--------------------
| Parameter        | Type          | Description                                                                                                   | Default              |
|-----------------|--------------|---------------------------------------------------------------------------------------------------------------|----------------------|
| `P`             | `int`        | Total number of ports.                                                                                        | **required**         |
| `C`             | `int`        | Total vessel capacity (number of containers that can be carried).                                             | **required**         |
| `target_utils`  | `list[float]`| Target utilization fractions per loading port. The length of target_utils should match the number of loading ports (P//2 if loading_only=True, otherwise P-1).                                                                | **required**         |
| `middle_leg`    | `int`        | Index separating loading and discharging ports.                                                               | `P // 2` if `None`   |
| `loading_only`  | `bool`       | If `True`, loading ports only ship to discharging ports.                                                      | `False`              |
| `sparsity`      | `float`      | Probability of zeroing an OD pair in the matrix, in `[0, 1]`.                                                 | `0.3`                |
| `perturb`       | `float`      | Fractional perturbation applied to matrix entries, in `[0, 1]`. Amount of random variation applied to each entry in the generated demand matrix.                                               | `0.2`                |
| `cargo_shares`  | `list`       | Shares for each cargo type (will be normalized). If `None`, uniform shares are used.                         | `None`               |
| `include_reefer`| `bool`       | Whether to include reefer cargo types.                                                                        | `True`               |
| `distribution`  | `str`        | Distribution used for randomization (`"poisson"`, `"neg_binomial"`, `"lognormal"`, `"normal"`, `"uniform"`).  | `"poisson"`          |
| `cv_demand`     | `float`      | Coefficient of variation of distribution. Common values `{0.5, 1.0, 1.5}`, but in `(0, ∞)`. It controls the relative amount of randomness (variation) in the generated demand scenarios.                  | `1.0`                |
| `seed`          | `int`        | Random seed for reproducibility.                                                                              | `None`               |


In [29]:
### Generate LD for the current port
P = 10
C = 7476
target_utils = np.ones(P-1)  # Length should match number of loading ports
current_port = 1
middle_leg = None
loading_only = False

distribution = "normal"

seed = 98364

dg = DemandGenerator(
    P=P,
    C=C,
    target_utils=target_utils,
    current_port=current_port,
    include_current_port=True,
    middle_leg=middle_leg,
    loading_only=loading_only,
    sparsity=0.0,
    perturb=0.1,
    distribution="normal",
    seed=seed
)

loading_list = dg.generate_loading_list()


### Generate demand scenarios for all sucessive ports
n_scenarios = 5     # Number of scenarios

# Example target_utils for loading_only=True (length = P//2)
target_utils = np.ones(P-1) * 0.95 # Length 6 for P=7 and loading_only=False
include_current_port = False
current_port_ld = loading_list

# Generate data for ports from start_port to start_port + P - 1
dg = DemandGenerator(
    P = P,
    C = C,
    target_utils = target_utils,
    current_port = current_port,
    include_current_port = False,
    current_port_ld = current_port_ld,
    middle_leg = None,
    loading_only = loading_only,
    sparsity = 0.0,
    perturb = 0.3,
    cargo_shares = None,
    distribution = distribution,
    cv_demand = 1.0,
    seed = seed
)

expected_demand, std_demand = dg._generate_moments()
scenarios = dg._generate(expected_demand, std_demand, n_scenarios=n_scenarios)

In [30]:
# Analyze onboard demand and utilization for the LD generated for the current port
ob_demand_ld = []
ob_teus_ld = []
def onboard_transports(ports, pol):
    return [(i, j) for i in range(ports) for j in range(ports) if i <= pol and j > pol]
for pol in range(P - 1):
    ob = 0
    ob_teu = 0
    k = 0
    for key in dg.cargo_types:
        for (oi, dj) in onboard_transports(P, pol):
            ob += loading_list[key][oi, dj]
            ob_teu += loading_list[key][oi, dj] * dg.teu[k]
        k += 1
    ob_demand_ld.append(int(ob))
    ob_teus_ld.append(int(ob_teu))
print("LD onboard demand per port:", ob_demand_ld)
print("LD onboard TEU per port:", ob_teus_ld)
actual_utils_ld = np.array(ob_teus_ld) / C
print("LD actual utilizations:", actual_utils_ld)
print("-" * 40)
print("-" * 40)

# Analyze onboard demand and utilization for each scenario
for i, scenario in enumerate(scenarios):
    print(f"Scenario {i+1}:")
    ob_demand = []
    ob_teus = []
    def onboard_transports(ports, pol):
        return [(i, j) for i in range(ports) for j in range(ports) if i <= pol and j > pol]
    for pol in range(P - 1):
        ob = 0
        ob_teu = 0
        for (oi, dj) in onboard_transports(P, pol):
            k = 0
            for key in dg.cargo_types:
                ob += scenario[key][oi, dj]
                ob_teu += scenario[key][oi, dj] * dg.teu[k]
                k += 1
        ob_demand.append(int(ob))
        ob_teus.append(int(ob_teu))
    print(f"  Onboard demand per port: {ob_demand}")
    print(f"  Onboard TEU per port: {ob_teus}")
    actual_utils = np.array(ob_teus) / C
    print(f"  Actual utilizations: {actual_utils}")
    print("-" * 40)

LD onboard demand per port: [4668, 4183, 3727, 3320, 2857, 2268, 1737, 1277, 704]
LD onboard TEU per port: [7320, 6519, 5880, 5204, 4489, 3591, 2738, 2006, 1064]
LD actual utilizations: [0.97913323 0.87199037 0.78651685 0.69609417 0.60045479 0.48033708
 0.36623863 0.26832531 0.1423221 ]
----------------------------------------
----------------------------------------
Scenario 1:
  Onboard demand per port: [0, 4517, 4633, 4469, 4418, 4404, 4291, 4310, 4554]
  Onboard TEU per port: [0, 6876, 7189, 6967, 6952, 6835, 6868, 6647, 6975]
  Actual utilizations: [0.         0.91974318 0.96161049 0.93191546 0.92990904 0.91425896
 0.91867309 0.88911182 0.93298555]
----------------------------------------
Scenario 2:
  Onboard demand per port: [0, 4019, 4289, 4105, 4177, 4226, 3899, 4318, 4957]
  Onboard TEU per port: [0, 6486, 6931, 6553, 6681, 6752, 6305, 6983, 7695]
  Actual utilizations: [0.         0.86757624 0.92710005 0.87653826 0.89365971 0.90315677
 0.84336544 0.93405564 1.02929374]
-----

In [ ]:
# Export loading_list to a .txt file
FileName_LD = f"S_{P}_{current_port}_{n_scenarios}_{loading_only}_CP.txt"

output_file_ld = FileName_LD
with open(output_file_ld, "w") as f:
    # Write first line: number of ports and 1 scenario (LD is deterministic)
    f.write(f"{P} 1\n")
    # Write container types (remove 'ft' from size)
    for ctype in dg.cargo_types:
        size, weight, ctype_str = ctype
        size_int = int(str(size).replace('ft', ''))
        f.write(f"{size_int} {weight} {ctype_str}\n")
    # Write the LD matrices
    for ctype in dg.cargo_types:
        matrix = loading_list[ctype]
        for row in matrix:
            f.write(" ".join(str(int(x)) for x in row) + "\n")
print(f"LD exported to {output_file_ld}")

# Data file name, automatically set based on number of ports and scenarios
FileName = f"S_{P}_{current_port}_{n_scenarios}_{loading_only}.txt"

# Export scenarios to a .txt file
output_file = FileName
with open(output_file, "w") as f:
    # Write first line: number of ports and number of scenarios
    f.write(f"{P} {n_scenarios}\n")
    # Write container types (remove 'ft' from size)
    for ctype in dg.cargo_types:
        size, weight, ctype_str = ctype
        size_int = int(str(size).replace('ft', ''))
        f.write(f"{size_int} {weight} {ctype_str}\n")
    # Write all scenario matrices
    for scenario in scenarios:
        for ctype in dg.cargo_types:
            matrix = scenario[ctype]
            for row in matrix:
                f.write(" ".join(str(int(x)) for x in row) + "\n")
print(f"Scenarios exported to {output_file}")

LD exported to S_10_1_5_False_CP_LD.txt
Scenarios exported to S_10_1_5_False.txt
